# Imports + paths

In [2]:
import pandas as pd 
import re

Variables needed: 
- MMR vaccination coverage (%)
- Serological immunity (%)
- Vaccine effectiveness (fixed for MMR)
- Effective immunity --> derived 
- Exemption rate (%)
- Annual measles cases
- Outbreak indicator --> derived
- Attack rate (cases / population) --> derived 
- Total population 
- Children (0-9) proportion 
- Population density
- Birth rate
- International travel volume 
- Prior global measles incidence
- Year
- Region 



# Data loading: County-level MMR vaccination coverage

In [3]:
mmr = pd.read_csv('../data/raw/mmr_data_us_counties.csv')

In [4]:
mmr

,fips,county,state,SY2016_17,SY2017_18,SY2018_19,SY2019_20,SY2020_21,SY2021_22,SY2022_23,SY2023_24
0,1001,Autauga,AL,0.9527,0.9639,0.9536,0.8866,0.8977,0.9201,0.9123,NaN
1,1003,Baldwin,AL,0.9794,0.9653,0.9700,0.9291,0.8804,0.9332,0.9257,NaN
2,1005,Barbour,AL,0.9169,0.8817,0.9283,0.9256,0.9342,0.9248,0.9459,NaN
3,1007,Bibb,AL,0.9657,0.9454,0.9432,0.9598,0.9438,0.9474,0.9122,NaN
4,1009,Blount,AL,0.9758,0.9730,0.9740,0.9762,0.9626,0.9765,0.9700,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2334,56037,Sweetwater,WY,NaN,NaN,NaN,NaN,NaN,NaN,0.9600,NaN
2335,56039,Teton,WY,NaN,NaN,NaN,NaN,NaN,NaN,0.9500,NaN
2336,56041,Uinta,WY,NaN,NaN,NaN,NaN,NaN,NaN,0.9200,NaN
2337,56043,Washakie,WY,NaN,NaN,NaN,NaN,NaN,NaN,0.9000,NaN


Melt wide "SY20XX-XX" format to long: 

In [5]:
year_cols = [c for c in mmr.columns if c.upper().startswith('SY')]

mmr_long = mmr.melt(
    id_vars=['fips', 'county', 'state'],
    value_vars=year_cols,
    var_name='school_year',
    value_name='mmr_coverage'
)


Extract numeric year from "SY20XX-XX": 

In [6]:
mmr_long['year'] = mmr_long['school_year'].str.extract(r'(\d{4})').astype(int)
mmr_long = mmr_long.drop(columns=['school_year'])
mmr_long

,fips,county,state,mmr_coverage,year
0,1001,Autauga,AL,0.9527,2016
1,1003,Baldwin,AL,0.9794,2016
2,1005,Barbour,AL,0.9169,2016
3,1007,Bibb,AL,0.9657,2016
4,1009,Blount,AL,0.9758,2016
...,...,...,...,...,...
18707,56037,Sweetwater,WY,NaN,2023
18708,56039,Teton,WY,NaN,2023
18709,56041,Uinta,WY,NaN,2023
18710,56043,Washakie,WY,NaN,2023


Check inferred data types: 

In [7]:
mmr_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18712 entries, 0 to 18711
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   fips          18712 non-null  int64  
 1   county        18712 non-null  object 
 2   state         18712 non-null  object 
 3   mmr_coverage  14207 non-null  float64
 4   year          18712 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 731.1+ KB


Enforce explicit data types: 

In [8]:
mmr_long = mmr_long.astype({'fips': 'string', 'county': 'string', 'state': 'string', 'year': 'int32', 'mmr_coverage': 'float32'})

In [9]:
mmr_long 

,fips,county,state,mmr_coverage,year
0,1001,Autauga,AL,0.9527,2016
1,1003,Baldwin,AL,0.9794,2016
2,1005,Barbour,AL,0.9169,2016
3,1007,Bibb,AL,0.9657,2016
4,1009,Blount,AL,0.9758,2016
...,...,...,...,...,...
18707,56037,Sweetwater,WY,NaN,2023
18708,56039,Teton,WY,NaN,2023
18709,56041,Uinta,WY,NaN,2023
18710,56043,Washakie,WY,NaN,2023


Sort and linearly interpolate between known years: 

In [10]:
mmr_long = mmr_long.sort_values(by=['state', 'county', 'year'])
mmr_long = mmr_long.infer_objects()
mmr_long['mmr_coverage'] = mmr_long.groupby(['state', 'county'])['mmr_coverage'].transform(lambda x: x.interpolate(method='linear', limit_area='inside'))
mmr_long

,fips,county,state,mmr_coverage,year
0,1001,Autauga,AL,0.9527,2016
2339,1001,Autauga,AL,0.9639,2017
4678,1001,Autauga,AL,0.9536,2018
7017,1001,Autauga,AL,0.8866,2019
9356,1001,Autauga,AL,0.8977,2020
...,...,...,...,...,...
9355,56045,Weston,WY,NaN,2019
11694,56045,Weston,WY,NaN,2020
14033,56045,Weston,WY,NaN,2021
16372,56045,Weston,WY,0.6300,2022


Apply Last Observation Carried Forward (LOCF) to small gaps at edges: 

In [11]:
mmr_long['mmr_coverage'] = mmr_long.groupby(['state', 'county'])['mmr_coverage'].transform(lambda x: x.ffill(limit=2).bfill(limit=2))
mmr_long 

,fips,county,state,mmr_coverage,year
0,1001,Autauga,AL,0.9527,2016
2339,1001,Autauga,AL,0.9639,2017
4678,1001,Autauga,AL,0.9536,2018
7017,1001,Autauga,AL,0.8866,2019
9356,1001,Autauga,AL,0.8977,2020
...,...,...,...,...,...
9355,56045,Weston,WY,NaN,2019
11694,56045,Weston,WY,0.6300,2020
14033,56045,Weston,WY,0.6300,2021
16372,56045,Weston,WY,0.6300,2022


# Data loading: County-level case counts

In [12]:
cases = pd.read_csv('../data/raw/case_counts.csv')

In [13]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    269 non-null    int64 
 1   state   269 non-null    object
 2   county  269 non-null    object
 3   fips    269 non-null    int64 
 4   count   269 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 10.6+ KB


In [14]:
cases = cases.astype({'year': 'int32', 'state': 'string', 'county': 'string', 'fips': 'string', 'count': 'int32'})

Create base DataFrame of state + county + FIPS + year: 

In [15]:
base = mmr_long[['state', 'county', 'fips', 'year']].drop_duplicates().reset_index(drop=True) 
base

,state,county,fips,year
0,AL,Autauga,1001,2016
1,AL,Autauga,1001,2017
2,AL,Autauga,1001,2018
3,AL,Autauga,1001,2019
4,AL,Autauga,1001,2020
...,...,...,...,...
18707,WY,Weston,56045,2019
18708,WY,Weston,56045,2020
18709,WY,Weston,56045,2021
18710,WY,Weston,56045,2022


Merge case counts with base via left-join and fill other rows with 0: 

In [16]:
m1 = base.merge(cases[['fips', 'year', 'count']], on=['fips', 'year'], how='left', suffixes=('', '_byfips'))

# Second pass to fill NaNs using state + county + year 
mask_name_merge = m1['count'].isna()
if mask_name_merge.any(): 
    m2 = m1.loc[mask_name_merge, ['state', 'county', 'year']].merge(cases[['state', 'county', 'year', 'count']], on=['state', 'county', 'year'], how='left')
    m1.loc[mask_name_merge, 'count'] = m2['count'].values

merged = m1
merged 

,state,county,fips,year,count
0,AL,Autauga,1001,2016,NaN
1,AL,Autauga,1001,2017,NaN
2,AL,Autauga,1001,2018,NaN
3,AL,Autauga,1001,2019,NaN
4,AL,Autauga,1001,2020,NaN
...,...,...,...,...,...
18707,WY,Weston,56045,2019,NaN
18708,WY,Weston,56045,2020,NaN
18709,WY,Weston,56045,2021,NaN
18710,WY,Weston,56045,2022,NaN


In [17]:
merged['count'] = merged['count'].fillna(0).astype('int32')
merged['had_cases'] = merged['count'] > 0   # Flag for convenience 


Join back to MMR vaccination coverage data: 

In [18]:
vax_merged = mmr_long.merge(merged[['state', 'county', 'fips', 'year', 'count', 'had_cases']], on=['state', 'county', 'fips', 'year'], how='left')
vax_merged

,fips,county,state,mmr_coverage,year,count,had_cases
0,1001,Autauga,AL,0.9527,2016,0,False
1,1001,Autauga,AL,0.9639,2017,0,False
2,1001,Autauga,AL,0.9536,2018,0,False
3,1001,Autauga,AL,0.8866,2019,0,False
4,1001,Autauga,AL,0.8977,2020,0,False
...,...,...,...,...,...,...,...
18707,56045,Weston,WY,NaN,2019,0,False
18708,56045,Weston,WY,0.6300,2020,0,False
18709,56045,Weston,WY,0.6300,2021,0,False
18710,56045,Weston,WY,0.6300,2022,0,False


In [19]:
vax_merged['mmr_coverage'].isna().sum()

np.int64(1063)

# Data loading: County characteristics resident population estimates (age structure data)

In [20]:
pop20 = pd.read_csv('../data/raw/CC-EST2020-AGESEX-ALL.csv', engine='python', 
                    usecols=['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'POPESTIMATE', 'UNDER5_TOT', 'AGE513_TOT', 'AGE1417_TOT', 'AGE1824_TOT', 'AGE2544_TOT', 'AGE4564_TOT', 'AGE65PLUS_TOT'])
pop24 = pd.read_csv('../data/raw/cc-est2024-agesex-all.csv', engine='python',
                    usecols=['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'POPESTIMATE', 'UNDER5_TOT', 'AGE513_TOT', 'AGE1417_TOT', 'AGE1824_TOT', 'AGE2544_TOT', 'AGE4564_TOT', 'AGE65PLUS_TOT'])


Filter on years 9-12 for 2020 estimates and add 7 + filter on yeras 2-5 for 2024 estimates and add 18: 

In [21]:
# Keep relevant years and reformat by adding offsets
pop20 = pop20[pop20['YEAR'].isin([9, 10, 11, 12])].copy()
pop20['YEAR'] = pop20['YEAR'] + 2007 
pop24 = pop24[pop24['YEAR'].isin([2, 3, 4, 5])].copy()
pop24['YEAR'] = pop24['YEAR'] + 2018
# Concatenate filtered dataframes
pop = pd.concat([pop20, pop24], ignore_index=True)

In [22]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25148 entries, 0 to 25147
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   STATE          25148 non-null  int64 
 1   COUNTY         25148 non-null  int64 
 2   STNAME         25148 non-null  object
 3   CTYNAME        25148 non-null  object
 4   YEAR           25148 non-null  int64 
 5   POPESTIMATE    25148 non-null  int64 
 6   UNDER5_TOT     25148 non-null  int64 
 7   AGE513_TOT     25148 non-null  int64 
 8   AGE1417_TOT    25148 non-null  int64 
 9   AGE1824_TOT    25148 non-null  int64 
 10  AGE2544_TOT    25148 non-null  int64 
 11  AGE4564_TOT    25148 non-null  int64 
 12  AGE65PLUS_TOT  25148 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 2.5+ MB


Coerce data types explicitly:

In [23]:
pop = pop.astype({'STATE': 'string', 'COUNTY': 'string', 'STNAME': 'string', 'CTYNAME': 'string', 'YEAR': 'int32',
                      'POPESTIMATE': 'int32', 'UNDER5_TOT': 'int32', 'AGE513_TOT': 'int32', 'AGE1417_TOT': 'int32', 'AGE1824_TOT': 'int32',
                      'AGE2544_TOT': 'int32', 'AGE4564_TOT': 'int32', 'AGE65PLUS_TOT': 'int32'})

Rename to standardize column names with existing dataframes: 

In [24]:
pop = pop.rename(columns={
    'STATE': 'state_fips',
    'COUNTY': 'county_fips',
    'STNAME': 'state',
    'CTYNAME': 'county',
    'YEAR': 'year',
    'POPESTIMATE': 'population',
    'UNDER5_TOT': 'pop_under5',
    'AGE513_TOT': 'pop_5to13',
    'AGE1417_TOT': 'pop_14to17',
    'AGE1824_TOT': 'pop_18to24',
    'AGE2544_TOT': 'pop_25to44',
    'AGE4564_TOT': 'pop_45to64',
    'AGE65PLUS_TOT': 'pop_65plus'
})

Reformat state and county FIPS codes into single value: 

In [25]:
# Pad county FIPS codes with leading zeros to ensure 3 digits
pop['county_fips'] = pop['county_fips'].str.zfill(3)
pop['fips'] = pop['state_fips'] + pop['county_fips']
pop

,state_fips,county_fips,state,county,year,population,pop_under5,pop_5to13,pop_14to17,pop_18to24,pop_25to44,pop_45to64,pop_65plus,fips
0,1,001,Alabama,Autauga County,2016,55302,3195,6705,3466,4624,14400,14819,8093,1001
1,1,001,Alabama,Autauga County,2017,55448,3286,6632,3336,4564,14406,14854,8370,1001
2,1,001,Alabama,Autauga County,2018,55533,3353,6490,3300,4435,14453,14935,8567,1001
3,1,001,Alabama,Autauga County,2019,55769,3251,6499,3199,4469,14635,14933,8783,1001
4,1,003,Alabama,Baldwin County,2016,207787,11714,22989,10786,15407,48857,57513,40521,1003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25143,56,043,Wyoming,Washakie County,2023,7736,350,776,450,530,1724,2001,1905,56043
25144,56,045,Wyoming,Weston County,2020,6817,310,690,342,443,1720,1853,1459,56045
25145,56,045,Wyoming,Weston County,2021,6747,296,693,326,458,1675,1824,1475,56045
25146,56,045,Wyoming,Weston County,2022,6872,290,691,323,497,1685,1832,1554,56045


In [26]:
pop['year'].unique()

array([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int32)

In [27]:
# 1) Compare unique key counts
left_keys = mmr_long[['state','county','fips','year']].drop_duplicates()
pop_keys  = pop[['state','county','fips','year']].drop_duplicates()  # or whichever pop variable you used
print("unique mmr_long keys:", len(left_keys))
print("unique pop keys   :", len(pop_keys))

# 2) Check year ranges
print("mmr_long years:", left_keys['year'].min(), "-", left_keys['year'].max())
print("pop years:     ", pop_keys['year'].min(), "-", pop_keys['year'].max())

# 3) Check missing FIPS / nulls in keys
for df,name in [(mmr_long,'mmr_long'), (pop,'pop')]:
    print(name, "null fips:", df['fips'].isna().sum(), "dtype:", df['fips'].dtype)
    print(name, "sample fips values:", df['fips'].dropna().astype(str).unique()[:10])

# 4) Check how many pop keys are NOT in mmr_long (these are rows that will be missing after an inner/left-by-mmr merge)
pop_not_in_mmr = pop_keys.merge(left_keys, on=['state','county','fips','year'], how='left', indicator=True)
print("pop keys missing in mmr_long:", (pop_not_in_mmr['_merge'] == 'left_only').sum())
pop_not_in_mmr.loc[pop_not_in_mmr['_merge']=='left_only'].head(10)

# 5) Show some examples of missing keys (helps spot patterns)
missing_examples = pop_not_in_mmr.loc[pop_not_in_mmr['_merge']=='left_only'].head(40)
missing_examples

unique mmr_long keys: 18712
unique pop keys   : 25148
mmr_long years: 2016 - 2023
pop years:      2016 - 2023
mmr_long null fips: 0 dtype: string
mmr_long sample fips values: ['1001' '1003' '1005' '1007' '1009' '1011' '1013' '1015' '1017' '1019']
pop null fips: 0 dtype: string
pop sample fips values: ['1001' '1003' '1005' '1007' '1009' '1011' '1013' '1015' '1017' '1019']
pop keys missing in mmr_long: 25148


,state,county,fips,year,_merge
0,Alabama,Autauga County,1001,2016,left_only
1,Alabama,Autauga County,1001,2017,left_only
2,Alabama,Autauga County,1001,2018,left_only
3,Alabama,Autauga County,1001,2019,left_only
4,Alabama,Baldwin County,1003,2016,left_only
5,Alabama,Baldwin County,1003,2017,left_only
6,Alabama,Baldwin County,1003,2018,left_only
7,Alabama,Baldwin County,1003,2019,left_only
8,Alabama,Barbour County,1005,2016,left_only
9,Alabama,Barbour County,1005,2017,left_only


Merge age structure data with vaccination coverage and case counts: 

In [28]:
df = vax_merged.merge(pop, on=['fips', 'year'], how='left')
df

,fips,county_x,state_x,mmr_coverage,year,count,had_cases,state_fips,county_fips,state_y,county_y,population,pop_under5,pop_5to13,pop_14to17,pop_18to24,pop_25to44,pop_45to64,pop_65plus
0,1001,Autauga,AL,0.9527,2016,0,False,1,001,Alabama,Autauga County,55302.0,3195.0,6705.0,3466.0,4624.0,14400.0,14819.0,8093.0
1,1001,Autauga,AL,0.9639,2017,0,False,1,001,Alabama,Autauga County,55448.0,3286.0,6632.0,3336.0,4564.0,14406.0,14854.0,8370.0
2,1001,Autauga,AL,0.9536,2018,0,False,1,001,Alabama,Autauga County,55533.0,3353.0,6490.0,3300.0,4435.0,14453.0,14935.0,8567.0
3,1001,Autauga,AL,0.8866,2019,0,False,1,001,Alabama,Autauga County,55769.0,3251.0,6499.0,3199.0,4469.0,14635.0,14933.0,8783.0
4,1001,Autauga,AL,0.8977,2020,0,False,1,001,Alabama,Autauga County,58909.0,3505.0,7049.0,3357.0,4589.0,15511.0,15613.0,9285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18707,56045,Weston,WY,NaN,2019,0,False,56,045,Wyoming,Weston County,6880.0,333.0,728.0,344.0,425.0,1662.0,1914.0,1474.0
18708,56045,Weston,WY,0.6300,2020,0,False,56,045,Wyoming,Weston County,6817.0,310.0,690.0,342.0,443.0,1720.0,1853.0,1459.0
18709,56045,Weston,WY,0.6300,2021,0,False,56,045,Wyoming,Weston County,6747.0,296.0,693.0,326.0,458.0,1675.0,1824.0,1475.0
18710,56045,Weston,WY,0.6300,2022,0,False,56,045,Wyoming,Weston County,6872.0,290.0,691.0,323.0,497.0,1685.0,1832.0,1554.0


In [29]:
# Z-fill state FIPS codes for width 2
df['state_fips'] = df['state_fips'].str.zfill(2)
# Fix FIPS codes that are not 5 characters long
df['fips'] = df['fips'].str.zfill(5)
df 

,fips,county_x,state_x,mmr_coverage,year,count,had_cases,state_fips,county_fips,state_y,county_y,population,pop_under5,pop_5to13,pop_14to17,pop_18to24,pop_25to44,pop_45to64,pop_65plus
0,01001,Autauga,AL,0.9527,2016,0,False,01,001,Alabama,Autauga County,55302.0,3195.0,6705.0,3466.0,4624.0,14400.0,14819.0,8093.0
1,01001,Autauga,AL,0.9639,2017,0,False,01,001,Alabama,Autauga County,55448.0,3286.0,6632.0,3336.0,4564.0,14406.0,14854.0,8370.0
2,01001,Autauga,AL,0.9536,2018,0,False,01,001,Alabama,Autauga County,55533.0,3353.0,6490.0,3300.0,4435.0,14453.0,14935.0,8567.0
3,01001,Autauga,AL,0.8866,2019,0,False,01,001,Alabama,Autauga County,55769.0,3251.0,6499.0,3199.0,4469.0,14635.0,14933.0,8783.0
4,01001,Autauga,AL,0.8977,2020,0,False,01,001,Alabama,Autauga County,58909.0,3505.0,7049.0,3357.0,4589.0,15511.0,15613.0,9285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18707,56045,Weston,WY,NaN,2019,0,False,56,045,Wyoming,Weston County,6880.0,333.0,728.0,344.0,425.0,1662.0,1914.0,1474.0
18708,56045,Weston,WY,0.6300,2020,0,False,56,045,Wyoming,Weston County,6817.0,310.0,690.0,342.0,443.0,1720.0,1853.0,1459.0
18709,56045,Weston,WY,0.6300,2021,0,False,56,045,Wyoming,Weston County,6747.0,296.0,693.0,326.0,458.0,1675.0,1824.0,1475.0
18710,56045,Weston,WY,0.6300,2022,0,False,56,045,Wyoming,Weston County,6872.0,290.0,691.0,323.0,497.0,1685.0,1832.0,1554.0


Add land area data and calculate population density: 

In [30]:
# Import CSV with land area data
area = pd.read_csv('../data/raw/us-county-areas.csv', names=['state_fips', 'county_fips', 'area_sqkm'])
area['area_sqkm'] = pd.to_numeric(area['area_sqkm'], errors='coerce')  
area = area.dropna(subset=['area_sqkm']).copy()
area['fips'] = area['state_fips'] + area['county_fips']
area['area_sqmi'] = area['area_sqkm'] / 2.58999
area = area[['fips', 'area_sqkm', 'area_sqmi']]
area 

,fips,area_sqkm,area_sqmi
1,31129,1491.364,575.818439
2,39003,1053.739,406.850606
3,39085,2536.109,979.196445
4,29011,1545.503,596.721609
5,26089,6558.347,2532.190086
...,...,...,...
3229,48001,2792.010,1078.000301
3230,51033,1391.431,537.234121
3231,46051,1780.703,687.532770
3232,42129,2683.451,1036.085468


In [31]:
# Merge area data into the main dataframe
df = df.merge(area, on='fips', how='left')
# Calculate population density
df['pop_density'] = df['population'] / df['area_sqkm']
df

,fips,county_x,state_x,mmr_coverage,year,count,had_cases,state_fips,county_fips,state_y,...,pop_under5,pop_5to13,pop_14to17,pop_18to24,pop_25to44,pop_45to64,pop_65plus,area_sqkm,area_sqmi,pop_density
0,01001,Autauga,AL,0.9527,2016,0,False,01,001,Alabama,...,3195.0,6705.0,3466.0,4624.0,14400.0,14819.0,8093.0,1565.309,604.368743,35.329766
1,01001,Autauga,AL,0.9639,2017,0,False,01,001,Alabama,...,3286.0,6632.0,3336.0,4564.0,14406.0,14854.0,8370.0,1565.309,604.368743,35.423038
2,01001,Autauga,AL,0.9536,2018,0,False,01,001,Alabama,...,3353.0,6490.0,3300.0,4435.0,14453.0,14935.0,8567.0,1565.309,604.368743,35.477340
3,01001,Autauga,AL,0.8866,2019,0,False,01,001,Alabama,...,3251.0,6499.0,3199.0,4469.0,14635.0,14933.0,8783.0,1565.309,604.368743,35.628109
4,01001,Autauga,AL,0.8977,2020,0,False,01,001,Alabama,...,3505.0,7049.0,3357.0,4589.0,15511.0,15613.0,9285.0,1565.309,604.368743,37.634103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18707,56045,Weston,WY,NaN,2019,0,False,56,045,Wyoming,...,333.0,728.0,344.0,425.0,1662.0,1914.0,1474.0,6216.030,2400.020850,1.106816
18708,56045,Weston,WY,0.6300,2020,0,False,56,045,Wyoming,...,310.0,690.0,342.0,443.0,1720.0,1853.0,1459.0,6216.030,2400.020850,1.096681
18709,56045,Weston,WY,0.6300,2021,0,False,56,045,Wyoming,...,296.0,693.0,326.0,458.0,1675.0,1824.0,1475.0,6216.030,2400.020850,1.085419
18710,56045,Weston,WY,0.6300,2022,0,False,56,045,Wyoming,...,290.0,691.0,323.0,497.0,1685.0,1832.0,1554.0,6216.030,2400.020850,1.105529


Drop NaN columns as final step before writing to CSV: 

In [32]:
# Drop rows with missing MMR coverage, case counts, or population estimates
df = df.dropna(subset=['mmr_coverage', 'count', 'population', 'pop_density'])
df.isna().sum()

fips            0
county_x        0
state_x         0
mmr_coverage    0
year            0
count           0
had_cases       0
state_fips      0
county_fips     0
state_y         0
county_y        0
population      0
pop_under5      0
pop_5to13       0
pop_14to17      0
pop_18to24      0
pop_25to44      0
pop_45to64      0
pop_65plus      0
area_sqkm       0
area_sqmi       0
pop_density     0
dtype: int64

Ensure valid MMR vaccination coverage values (0-1): 

In [33]:
# Ensure valid MMR vaccination coverage values (0-1):
df = df[(df['mmr_coverage'] >= 0) & (df['mmr_coverage'] <= 1)]
df.describe()

,mmr_coverage,year,count,population,pop_under5,pop_5to13,pop_14to17,pop_18to24,pop_25to44,pop_45to64,pop_65plus,area_sqkm,area_sqmi,pop_density
count,17337.000000,17337.000000,17337.000000,1.733700e+04,17337.000000,17337.000000,17337.000000,17337.000000,1.733700e+04,1.733700e+04,17337.000000,17337.000000,17337.000000,17337.000000
mean,0.926150,2019.690892,0.104055,1.157073e+05,6772.127588,13010.648440,6006.102728,10643.087904,3.073857e+04,2.930144e+04,19235.316087,2758.572721,1065.090105,93.888222
std,0.071956,2.208978,4.846134,3.202683e+05,19651.058430,36875.045108,16804.748912,29226.966727,9.239124e+04,7.958342e+04,48318.516636,3728.301262,1439.504115,426.195614
min,0.000000,2016.000000,0.000000,2.150000e+02,11.000000,31.000000,8.000000,8.000000,4.700000e+01,5.500000e+01,47.000000,5.300000,2.046340,0.067664
25%,0.907000,2018.000000,0.000000,1.195600e+04,648.000000,1287.000000,609.000000,896.000000,2.663000e+03,3.146000e+03,2409.000000,1273.539000,491.715798,6.516486
50%,0.947800,2020.000000,0.000000,2.787500e+04,1559.000000,3040.000000,1427.000000,2249.000000,6.559000e+03,7.341000e+03,5463.000000,1818.331000,702.061012,16.247960
75%,0.971459,2022.000000,0.000000,7.767500e+04,4285.000000,8576.000000,4046.000000,7128.000000,1.866700e+04,2.015800e+04,14547.000000,2667.154000,1029.793165,44.926776
max,1.000000,2023.000000,576.000000,5.267072e+06,355811.000000,635302.000000,295501.000000,475379.000000,1.575305e+06,1.306353e+06,846376.000000,52071.995000,20105.095000,10832.456035


Convert age group totals to proportions that sum to 1: 

In [34]:
age_cols = ['pop_under5', 'pop_5to13', 'pop_14to17', 'pop_18to24', 'pop_25to44', 'pop_45to64', 'pop_65plus']
df[age_cols] = df[age_cols].div(df['population'], axis=0)

In [35]:
df.to_csv('../data/processed/vax_pop_merged.csv', index=False)